In [2]:
import time
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import csv
import pickle
import numpy as np
import pandas as pd
import pickle

from multiprocessing import Process
from multiprocessing import Pool
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

In [3]:

options = Options()

options.add_argument('--user-data-dir=/Users/Administrator/AppData/Local/Google/Chrome/User Data/') 
options.add_argument('--profile-directory=Profile 1')
options.add_argument("--disable-extensions")
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--start-maximized")

options.add_argument('disable-infobars')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument("--remote-debugging-port=9222")
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
options.add_argument(f'user-agent={user_agent}')

prefs = {
    "profile.default_content_setting_values.images": 2
}
options.add_experimental_option("prefs", prefs)

# Add a preference to disable image loading


capabilities = DesiredCapabilities.CHROME.copy()
capabilities['acceptSslCerts'] = True
capabilities['acceptInsecureCerts'] = True




In [4]:
file_path = 'categories.txt'

categories = []

with open(file_path, 'r') as file:
    for line in file:     
        category = line.strip()
        categories.append(category)

In [20]:
all_details = {}
index = 0
urls = []
for count,category in enumerate(categories):
    
    driver = webdriver.Chrome(options=options , desired_capabilities=capabilities)
    driver.get("https://dir.indiamart.com/search.mp?ss=scrap&prdsrc=1&stype=attr=1|attrS&res=RC4&qu-comp=to")
    
    input_section = driver.find_element(By.XPATH,"//*[@id='search_string']")
    input_section.clear()
    time.sleep(3)
    input_section.send_keys(category)
    time.sleep(5)
    input_section.send_keys(Keys.ENTER)
    time.sleep(4)


    soup_old = ""
    while True:
        print(f'Category:{category}----Category Number:{count+1}/{len(categories)}----Dict_Length:{len(all_details)}',end='\r')
        html_text = driver.page_source
        soup = BeautifulSoup(html_text,"lxml")

        products = soup.find("div",class_='right-group flx1 df fww pr')
        sections = products.find_all("section")

        for section in sections:
            details={}
            try:
                name = section.find("a",class_="prd-name").text
                price = section.find("span",class_="elps elps2 fs14 tac clr7").text
                url = section.find("a",class_="prd-name")["href"].split(" ")[0]
                if url in urls:
                    continue

                details["name"]= name
                details["price"] = price
                details["url"] = url
                details["category"] = category
                
                urls.append(url)
                all_details[index] = details
                index+=1
            except:
                
                try:
                    name = section.find("span",class_="noPdp prd-name").text
                    price = section.find("span",class_="elps elps2 fs14 tac clr7").text
                    url = None
                    details["name"]= name
                    details["price"] = price
                    details["url"] = url
                    details["category"] = category
                    all_details[index] = details
                    index+=1
                except Exception as e:
                    
                    continue

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.implicitly_wait(10)
        
        if soup_old == soup:
            break
        else:
            
            soup_old = soup
        try:
            btn = driver.find_element(By.XPATH, "//*[@id='app']/div/main/div[4]/div[2]/div[2]/span")

            driver.execute_script("arguments[0].click();",btn)

        except:
            break
    driver.close()

In [21]:
df = pd.DataFrame().from_dict(all_details).T
df = df.drop_duplicates()
df = df.reset_index()
df.to_csv(index=False)

'index,name,price,url,category\r\n0,Inconel 690 Scrap,₹550/ Kilogram,https://www.indiamart.com/proddetail/inconel-690-scrap-11643041197.html?pos=1&kwd=industrial,Industrial Scrap\r\n1,Titanium Scrap,₹250/ Kilogram,https://www.indiamart.com/proddetail/titanium-scrap-11674300391.html?pos=2&kwd=industrial,Industrial Scrap\r\n2,"Stainless Steel Plate Cut Scrap, 50 KG, Packaging Type: GUNNY BAGS",₹100/ Kg,https://www.indiamart.com/proddetail/stainless-steel-plate-cut-scrap-19281601388.html?pos=3&kwd=industrial,Industrial Scrap\r\n3,"Black Mild Steel Scrap, For Metal Industry, Packaging Type: Loose",₹43/ Kg,https://www.indiamart.com/proddetail/mild-steel-scrap-26010043330.html?pos=5&kwd=industrial,Industrial Scrap\r\n4,"Aluminium Scrap Lithium Battery, Model Name/Number: 18650/2100/32650, Battery Capacity: 500 mAh",₹90/ Kg,https://www.indiamart.com/proddetail/scrap-lithium-battery-24229959212.html?pos=6&kwd=industrial,Industrial Scrap\r\n5,"MS Copper Wire Scrap, For Industrial",₹105/ Kg,http

In [22]:
all_details

{0: {'name': 'Inconel 690 Scrap',
  'price': '₹550/ Kilogram',
  'url': 'https://www.indiamart.com/proddetail/inconel-690-scrap-11643041197.html?pos=1&kwd=industrial',
  'category': 'Industrial Scrap'},
 1: {'name': 'Titanium Scrap',
  'price': '₹250/ Kilogram',
  'url': 'https://www.indiamart.com/proddetail/titanium-scrap-11674300391.html?pos=2&kwd=industrial',
  'category': 'Industrial Scrap'},
 2: {'name': 'Stainless Steel Plate Cut Scrap, 50 KG, Packaging Type: GUNNY BAGS',
  'price': '₹100/ Kg',
  'url': 'https://www.indiamart.com/proddetail/stainless-steel-plate-cut-scrap-19281601388.html?pos=3&kwd=industrial',
  'category': 'Industrial Scrap'},
 3: {'name': 'Black Mild Steel Scrap, For Metal Industry, Packaging Type: Loose',
  'price': '₹43/ Kg',
  'url': 'https://www.indiamart.com/proddetail/mild-steel-scrap-26010043330.html?pos=5&kwd=industrial',
  'category': 'Industrial Scrap'},
 4: {'name': 'Aluminium Scrap Lithium Battery, Model Name/Number: 18650/2100/32650, Battery Capac